In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision import models

import warnings
warnings.filterwarnings("ignore")

In [27]:
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image
import cv2

In [2]:
from win32com.shell import shell, shellcon
import os

desktop_path = shell.SHGetKnownFolderPath(shellcon.FOLDERID_Desktop)
train_dir = os.path.join(desktop_path, "flora")
valid_dir = os.path.join(desktop_path, "valid")
test_dir = os.path.join(desktop_path, "test")
print("Train Directory:",os.listdir(train_dir))
print("Test Directory: ", os.listdir(test_dir))
print("Valid Directory: ", os.listdir(valid_dir))
CLASS_NAMES = os.listdir(train_dir)

Train Directory: ['Aloe', 'Venerina']
Test Directory:  ['testa', 'testv']
Valid Directory:  ['valida', 'validv']


In [3]:
print("Training data 1:",len(os.listdir(train_dir+'/'+'Aloe')))
print("Training data 2:",len(os.listdir(train_dir+'/'+'Venerina')))
print("Testing data 1:",len(os.listdir(test_dir+'/'+'testv')))
print("Testing data 2:",len(os.listdir(test_dir+'/'+'testa')))
print("Validation data 1:",len(os.listdir(valid_dir+'/'+'validv')))
print("Validation data 2:",len(os.listdir(valid_dir+'/'+'valida')))

Training data 1: 396
Training data 2: 302
Testing data 1: 16
Testing data 2: 16
Validation data 1: 15
Validation data 2: 16


In [4]:
!pip install imgaug

   ---------------------------------------- 0.0/948.0 kB ? eta -:--:--
   -- ------------------------------------- 61.4/948.0 kB 3.2 MB/s eta 0:00:01
   --------- ------------------------------ 225.3/948.0 kB 2.8 MB/s eta 0:00:01
   -------------- ------------------------- 348.2/948.0 kB 2.7 MB/s eta 0:00:01
   ------------------------------- -------- 757.8/948.0 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 948.0/948.0 kB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
    --------------------------------------- 0.3/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 6.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.5/12.8 MB 13.2 MB/s eta 0:00:01
   ----- ---------------------------------- 1.8/12.8 MB 11.2 MB/s eta 0:00:01
   ------ --------------------------------- 2.0/12.8 MB 9.7 MB/s eta 0:00:02
   ------ --------------------------------- 2.2/12.8 MB 9.4 MB/s eta 0:00:02
   

In [8]:
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa

In [38]:
ia.seed(1)

# Example batch of images.
# The array has shape (32, 64, 64, 3) and dtype uint8.

images = np.array(
    [ia.quokka(size=(64, 64)) for _ in range(32)],
    dtype=np.uint8
)

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order



In [39]:

foldername_aloe = os.path.join(train_dir,'Aloe')
for filename in os.listdir(foldername_aloe):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
    # Загрузка изображения
        image_path = os.path.join(foldername_aloe, filename)
        image = cv2.imread(image_path)
        
        # Применение аугментаций к изображению
        augmented_image = seq(image=image)
        
        # Сохранение аугментированного изображения
        augmented_filename = 'augmented_' + filename
        augmented_image_path = os.path.join(foldername_aloe, augmented_filename)
        cv2.imwrite(augmented_image_path, augmented_image)

print("Аугментация завершена.")


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations